In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# from utils.notebook_util import NotebookUtil, Prompter

from notebook_util import NotebookUtil, Prompter

/home/uns/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
dataset_name = 'flowbird-april--flan-alpaca-gpt4-xl'


# model_name = 'google/flan-t5-xl'
model_name = 'declare-lab/flan-alpaca-gpt4-xl'


question_prompter = Prompter(template_name="generate_question_t5", runner="local")
answer_prompter = Prompter(template_name="generate_answer_t5", runner="local")

dataset = NotebookUtil.load_dataset(dataset_name)

text_cases = NotebookUtil.load_raw_data_text()

WE ARE FLOWBIRD
Flowbird offers end to end Smart mobility, Parking and Electric Vehicle charging solutions. We provide towns and cities with the tools to make mobility in urban areas simple, multimodal and environmentally friendly, and the ability to understand and manage this mobility.
Partnering with local administrations and operators for over 65 years, we provide apps, terminals, kerbside management, enforcement - and the software to manage these remotely. We upgrade existing physical terminals to digital terminals with new services and tariffs and open them up to integrate third party apps. Flowbird orchestrates sales, payments, reporting, invoices and fines across multiple endpoints, also providing the data insights needed to inform decisions.

---

FLOWBIRD global payment platform manages more than 300M banking transactions a year. Over 100 million users per week in 80+ countries trust Flowbird's expertise, R&D and technology partnerships to reduce air pollution, provide EV char

In [6]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

base_model = T5ForConditionalGeneration.from_pretrained(
    model_name
).to("cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.93s/it]


In [ ]:
!nvidia-smi

In [14]:
q_prompt = question_prompter.generate_prompt(
    '',
    text_cases[0]
)
output = NotebookUtil.t5_generate(q_prompt, tokenizer, base_model)
print(output)

1. What are the key features of Flowbird's Smart mobility solutions? 2. How does Flowbird work with local administrations and operators to provide end-to-end Smart mobility solutions? 3. How does Flowbird upgrade existing physical terminals to digital terminals with new services and tariffs?


# TODO

* make new quesiton splitter

max new token 1000
top-p 0.95
top-l 50
temperature 1